In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *


## Connect to Database

In [ ]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)

In [5]:
user = getpass.getuser()
#password = getpass.getpass("Password:")

user = "neo4j"
password = 'GREYSTONE!!'

database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

### get CSV information

In [ ]:
csv_file = os.path.join(project_path ,'data', 'gs_data', 'gs_tool_tag_export.csv')
#csv_file = os.path.join(project_path ,'data', 'test.csv')

dataframe = pd.read_csv(csv_file)
dataframe.fillna("", inplace=True)

#csvSchemas_file = os.path.join(project_path , 'data', 'testCSVheader.yaml')
csvSchemas_file = os.path.join(project_path , 'data',  'gs_data', 'gs_header.yaml')

csvSchemas_dict = openYAMLFile(csvSchemas_file)

### Clean the database

In [ ]:
database.deleteData()
database.dropIndexes()
database.createIndexes()

## Store the data

In [ ]:
import time
start_time = time.time()
done = graphDatabase_from_TaggedCSV(database, dataframe, csvSchemas_dict)
print("--- %s seconds ---" % (time.time() - start_time))

# KPI

## Create the object for the KPI

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *

databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)

/Users/sam11/Git/ml-py
yaml file open


In [7]:
m = MachineKpi(operator="=", name = "h1", databaseInfo=databaseSchema_dict)
t = TechnicianKpi(operator=".-", name="steve", result="name", databaseInfo=databaseSchema_dict)
i = TagItemKpi(operator="<>", keyword="valve", result="keyword", databaseInfo=databaseSchema_dict)

set()
{'(issue:ISSUE)-[:COVERED]->(machine:MACHINE)'}
set()
{'(technician:HUMAN:TECHNICIAN)'}
set()
{'(tag_item:TAG:ONE_GRAM:ITEM)'}


In [11]:
kpi = m+t+i
print(kpi.cypher_createQuery())

set()
{'(technician:HUMAN:TECHNICIAN)', '(issue:ISSUE)-[:COVERED]->(machine:MACHINE)'}
set()
{'(tag_item:TAG:ONE_GRAM:ITEM)', '(technician:HUMAN:TECHNICIAN)', '(issue:ISSUE)-[:COVERED]->(machine:MACHINE)'}
MATCH (tag_item:TAG:ONE_GRAM:ITEM) ,(technician:HUMAN:TECHNICIAN) ,(issue:ISSUE)-[:COVERED]->(machine:MACHINE)
WHERE machine.name = "h1" AND technician.name STARTS WITH "steve" AND tag_item.keyword <> "valve"
RETURN DISTINCT tag_item.keyword, technician.name


In [9]:
done, filterResult = database.runQuery(kpi.cypher_createQuery())

dataframe = resultToObservationDataframe(filterResult)
dataframe

2028it [00:00, 179786.28it/s]


,tag_item.keyword,technician.name
0,chain,steve andreozzi
1,chain,steve c
2,chain,steve salley
3,feeder,steve andreozzi
4,feeder,steve c
5,feeder,steve salley
6,feeder chain,steve andreozzi
7,feeder chain,steve c
8,feeder chain,steve salley
9,machine,steve andreozzi
